<a href="https://colab.research.google.com/github/woodychang0611/EMNIST/blob/master/EMNIST_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Environment
If running on colab mount google drive, otherwise use 200g drive from NCCU GPU cloud

In [1]:
import sys
import os
if ('google.colab' in sys.modules):
  from google.colab import drive
  drive.mount('/content/gdrive')
  gdrive_root = 'gdrive/My Drive/Deep_Learning/'
  dataset_path = os.path.join(gdrive_root,'Dataset')
else:
  dataset_path = '200g/Dataset'
  pass

if not os.path.exists(dataset_path):
  raise Exception(f'dataset_path "{dataset_path}"" does not exist')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Load Dataset

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim
import torch.autograd
import torchvision
import torchvision.transforms
import numpy as np
from matplotlib import pyplot as plt
import datetime

trans = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.5,), (1.0,))])
use_cuda = None
if torch.cuda.is_available():
  print("CUDA available")
  use_cuda = True
else:
  print ("CUDA not available")
  use_cuda = False
if os.path.exists(dataset_path):
  train_set = torchvision.datasets.EMNIST(root=dataset_path, transform=trans,train =True, split="byclass",download=True)
  test_set = torchvision.datasets.EMNIST(root=dataset_path, transform=trans,split="byclass", train =False)
  print (f'Dataset loaded, {train_set.__len__():,} training set, {test_set.__len__():,} testing set')
else:
  print (f'dataset_path "{dataset_path}" not found')
  exit(0)

batch_size = 128

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)


CUDA available
Dataset loaded, 697,932 training set, 116,323 testing set


# Common Functions

In [0]:
ceriation = nn.CrossEntropyLoss()
def apply_model(model,x,target):
    x, target = torch.autograd.Variable(x), torch.autograd.Variable(target)
    out = model(x)
    loss = torch.nn.CrossEntropyLoss()(out, target)
    return out,loss
def plot_graph(name,train_loss,test_loss):
  fig, ax = plt.subplots()
  ax.plot(train_loss,label='train loss')
  ax.plot(test_loss,label='test loss')  
  ax.set(xlabel='epoch', ylabel='Loss',title=name)
  plt.legend()
  plt.show()


# Define DNN

In [0]:
#DNN
class BaselineNet(nn.Module):
    def __init__(self):
        super(BaselineNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 62)
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
class ImprovedeNet(nn.Module):
    def __init__(self,dropout=0,batch_normalization=False,width=512):
        super(ImprovedeNet, self).__init__()
        self.dropout=dropout
        self.batch_normalization=batch_normalization
        self.fc1 = nn.Linear(28*28, width)
        self.bn1 = nn.BatchNorm1d(num_features=width)
        self.fc2 = nn.Linear(width, 256)
        self.bn2 = nn.BatchNorm1d(num_features=256)
        self.fc3 = nn.Linear(256, 62)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.dropout(x,self.dropout)
        x = self.fc1(x)
        if self.batch_normalization:
            x = self.bn1(x)
        x = F.relu(x)
        x = self.fc2(x)
        if self.batch_normalization:
            x = self.bn2(x)
        x = F.relu(x)
        x = F.dropout(x,self.dropout)
        x = self.fc3(x)
        return x

In [0]:
models = (
    ("Improved Net width =700",ImprovedeNet(width = 700, batch_normalization=False)),
    ("Improved Net width = 1000",ImprovedeNet(width=1000, batch_normalization=False)),    
    )
sample_limit = 100000000
for (name, model) in models:
  print(datetime.datetime.now())
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
  parameter_len = sum(p.numel() for p in model.parameters() if p.requires_grad)
  print(f'model:{name}, parameter:{parameter_len:,}')
  train_loss=[]
  test_loss=[]
  if use_cuda:
    model.cuda()
  for epoch in range(30):
    # training
    ave_loss = 0
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        out,loss = apply_model (model,x,target)
        ave_loss = ave_loss * 0.9 + loss.data* 0.1
        loss.backward()
        optimizer.step()
        if (batch_idx+1) == len(train_loader) or batch_idx >sample_limit:
            print (f'==>>> epoch: {epoch}, batch index: {batch_idx+1}, train loss: {ave_loss:.6f}')
            break
    train_loss.append(ave_loss)
    # testing
    correct_cnt, ave_loss = 0, 0
    total_cnt = 0
    for batch_idx, (x, target) in enumerate(test_loader):
        if use_cuda:
          x, target = x.cuda(), target.cuda()      
        out,loss = apply_model (model,x,target)        
        _, pred_label = torch.max(out.data, 1)
        total_cnt += x.data.size()[0]
        correct_cnt += (pred_label == target.data).sum()
        # smooth average
        ave_loss = ave_loss * 0.9 + loss.data * 0.1
        if (batch_idx) == len(test_loader)-1 or batch_idx >sample_limit:
            print (f'==>>> epoch: {epoch}, batch index: {batch_idx+1}, test loss: {ave_loss:.6f}, acc: {correct_cnt * 1.0 / total_cnt:.3f}')
            break
    test_loss.append(ave_loss)
  plot_graph(name,train_loss,test_loss)    

2020-03-31 13:26:25.864519
model:Improved Net width =700, parameter:746,802
==>>> epoch: 0, batch index: 5453, train loss: 0.635100
==>>> epoch: 0, batch index: 909, test loss: 0.572262, acc: 0.809
==>>> epoch: 1, batch index: 5453, train loss: 0.482484
==>>> epoch: 1, batch index: 909, test loss: 0.483241, acc: 0.831


In [16]:
2020-03-31 11:03:30.496378
model:Baseline Net, parameter:536,690
==>>> epoch: 0, batch index: 5453, train loss: 0.609612
==>>> epoch: 0, batch index: 909, test loss: 0.585146, acc: 0.809
==>>> epoch: 1, batch index: 5453, train loss: 0.540112
==>>> epoch: 1, batch index: 909, test loss: 0.493715, acc: 0.827
==>>> epoch: 2, batch index: 5453, train loss: 0.454728
==>>> epoch: 2, batch index: 909, test loss: 0.459707, acc: 0.839
==>>> epoch: 3, batch index: 5453, train loss: 0.453971
==>>> epoch: 3, batch index: 909, test loss: 0.450110, acc: 0.843
==>>> epoch: 4, batch index: 5453, train loss: 0.449814
==>>> epoch: 4, batch index: 909, test loss: 0.446976, acc: 0.844
==>>> epoch: 5, batch index: 5453, train loss: 0.424157
==>>> epoch: 5, batch index: 909, test loss: 0.423205, acc: 0.847
==>>> epoch: 6, batch index: 5453, train loss: 0.400469
==>>> epoch: 6, batch index: 909, test loss: 0.418983, acc: 0.850
==>>> epoch: 7, batch index: 5453, train loss: 0.387716
==>>> epoch: 7, batch index: 909, test loss: 0.417940, acc: 0.852
==>>> epoch: 8, batch index: 5453, train loss: 0.356984
==>>> epoch: 8, batch index: 909, test loss: 0.400661, acc: 0.854
==>>> epoch: 9, batch index: 5453, train loss: 0.410329
==>>> epoch: 9, batch index: 909, test loss: 0.411788, acc: 0.852


SyntaxError: ignored